In [1]:
import pandas as pd
import fundamentus as fund
import yfinance as yf

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [2]:
def filterTheDict(dictObj, callback):
    newDict = dict()
    # Iterate over all the items in dictionary
    for (key, value) in dictObj.items():
        # Check if item satisfies the given condition then add to new dict
        if callback((key, value)):
            newDict[key] = value
    return newDict

In [3]:
%%time

lista = fund.get_data()

100%|██████████| 426/426 [03:45<00:00,  1.89it/s]Wall time: 3min 45s



In [4]:
%%time 

lista.groupby('setor')['codigo'].count().reset_index(name='qtd acoes').sort_values(['qtd acoes'], ascending=False) 

Wall time: 3 ms


,setor,qtd acoes
12,Energia Elétrica,67
18,Intermediários Financeiros,37
34,"Tecidos, Vestuário e Calçados",28
8,Construção Civil,25
6,Comércio,24
14,Exploração de Imóveis,16
33,Siderurgia e Metalurgia,16
24,Máquinas e Equipamentos,15
27,Previdência e Seguros,15
36,Transporte,13


In [5]:
%%time 

stock_list = lista[['setor', 'codigo', 'cotacao', 'pl', 'roe', 'div_yield', 'lpa']].sort_values(['setor'], ascending=False).reset_index()
stock_list['y_codigo'] = (stock_list.codigo + '.SA')

Wall time: 25 ms


In [6]:
%%time 

stock_data = yf.download(list(stock_list['y_codigo']),  period = "2y")
stock_closed = stock_data['Close']

[*********************100%***********************]  426 of 426 completed
Wall time: 18.2 s


In [7]:
%%time 

mu = mean_historical_return(stock_closed)
S = CovarianceShrinkage(stock_closed).ledoit_wolf()

Wall time: 1.13 s


In [8]:
%%time 

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

Expected annual return: 158.6%
Annual volatility: 62.1%
Sharpe Ratio: 2.52
Wall time: 2.59 s


(1.5861335552137, 0.6207610945622705, 2.522924791731767)

In [ ]:
%%time 

latest_prices = get_latest_prices(stock_closed)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1200)
allocation, leftover = da.lp_portfolio()
print(allocation)
print(leftover)

In [9]:
%%time 

chosen_stocks = filterTheDict(cleaned_weights, lambda elem: elem[1] > 0)
chosen_stocks

Wall time: 1.03 ms


In [35]:
# import numpy as np
# prices = np.isnan(latest_prices)

In [37]:
# prices[prices == True]

PDTC3.SA    True
Name: 2021-03-04 00:00:00, dtype: bool